# Preprocessing เพื่อหา label ให้ชุดข้อมูล

## Install package & Import library

In [2]:
%%capture
# dev version
!pip install https://github.com/PyThaiNLP/pythainlp/archive/dev.zip

# release version 
! pip install pythainlp
!pip install epitran
!pip install sklearn_crfsuite
!pip install tensorflow deepcut
!pip install attacut
!pip install emoji
!pip install pytcc

In [3]:
import tweepy
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import emoji
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.tokenize import tcc 
from wordcloud import WordCloud
import matplotlib.pyplot as plt 
from pythainlp.corpus import thai_stopwords
import re
import json
import pytcc
import subprocess
import os

###JSON

In [ ]:
with open('cat_subcat.json',encoding='utf-8') as files:
  dictcatsubcat = json.load(files)
dictcatsubcat

## เหล่า Function

In [22]:
#วนไฟล์เข้า
def walk_csv(directory:str):
    root, dirs, files = list(os.walk(directory))[0]
    return sorted([os.path.join(directory, file) for file in files if file.endswith('.csv') ])
#แยกพนักงาน
def check_agent(df):
  df0 = df.dropna()
  df0 = df0.drop(df0[df0.speaker == '-'].index)
  df0 = df0.drop(df0[df0.speaker == 'Overlap'].index)
  for i in df0['transcripts']:
    if (any( w in i for w in ['สวัสดี','ประกัน','ภัย','วิริยะ'] )) and (any( w in i for w in ['วิริยะ','ยินดี','บริการ'] )) :
      agent = list(df0.speaker[df0[df0.transcripts == i].index])[0]
      break
    if (all( w in i for w in ['ตรวจสอบ','ขออนุญาต'] )) :
      agent = list(df0.speaker[df0[df0.transcripts == i].index])[0]
      break
    elif ('เรียนสาย') in i:
      agent = list(df0.speaker[df0[df0.transcripts == i].index])[0]
      break
    elif ('รับสาย') in i:
      agent = list(df0.speaker[df0[df0.transcripts == i].index])[0]
      break
    else:
      agent = ''
  lis = list(df0['speaker'].unique())
  lis.remove(agent)
  re_df0 = df0.replace(agent,'agent')
  if len(lis) < 2:
    re_df0 = re_df0.replace(lis[0] , 'customer')
  else:
    if len(df0[df0.speaker == lis[0]]) < 4:
      for a in lis:
        re_df0 = re_df0.replace(a,'customer')
    else:
      customer = lis[0]
      re_df0 = re_df0.replace(lis[0],'customer')
      lis.remove(customer)
      for a in lis:
        re_df0 = re_df0.replace(a,'agent')   
  return re_df0

In [6]:
#ทำความสะอาดประโยค
def cleanText(df):
  new_text = []
  for text in df['transcripts']:
    text = str(text)
    text = re.sub('[^ก-๙]','',text)
    new_text.append(text)
  df['transcripts_cleaning'] = new_text
  return df

In [7]:
#ไว้หาlabel
def getcat(text,lookup= dictcatsubcat ):
  for cat in lookup :
    for subcat in lookup[cat]:
      for keyword in lookup[cat][subcat]:
        if keyword in text:
          return {"cat":cat,"subcat":subcat}
  return {"cat":"Service Dialogue","subcat":"Repeat Information"}

In [48]:
#แก้ปัญหาการทับกันของ greeting,ending
def ending(df):
  for i in df.iloc[8:]['subcat']:
    if i == 'Greeting':
      df.iloc[8:]['subcat']=df.iloc[8:]['subcat'].str.replace(i,"Ending")
  return df

def listening(df):
  for i in df['subcat']:
    for index, row in df.iterrows():
      if len(row["transcripts_cleaning"]) < 7 :
        df.loc[index,'subcat']=row['subcat'].replace(i,'Listening')
  return df

## นำเข้าไฟล์ข้อมูล

###Import Data ยังไม่มี label

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
folder_path = '/content/drive/MyDrive/iiv/1557menu2/parsed'
file_path = walk_csv(folder_path)
file_path

In [ ]:
list_df = []
file_name = []
for i in file_path :
  files_list_text = i[44:-4]
  file_name.append(files_list_text)
  print(files_list_text)
  df_file = pd.read_csv(i, index_col=None, header=0)
  list_df.append(df_file)

In [27]:
len(list_df)

85

In [52]:
list_df[1]

,start,end,speaker,gender,transcripts
0,0.703,4.703,A,female,สวัสดีค่ะบริษัทวิริยะประกันภัยณิชาภายินดีให้บร...
1,5.509,6.919,B,female,อ๋อ สวัสดีค่ะ
2,22.592,25.728,A,female,สวัสดีค่ะตอนนี้ณิชาภาเรียนสายคุณอะไรนะคะ
3,29.016,31.984,A,female,ตอนนี้มีเจ้าหน้าที่ไปพบแล้วหรือยังคะ
4,33.245,42.859,B,female,อ๋อไม่ มันเป็นบริการออนไลน์อ่ะ แต่เขาก็ให้คำอธ...
...,...,...,...,...,...
62,188.950,190.078,A,female,ค่ะ
63,190.078,191.565,B,female,ซ่อมซ่อมนานไหมไรอย่างงี้
64,191.565,192.650,A,female,ค่ะ ค่ะ
65,193.304,194.554,B,female,ขอบคุณค่ะ


##loop label cat/subcat

In [57]:
error = []
for i in range(len(list_df)):
  try:
    df = check_agent(list_df[i])
    df = df.drop(df[df.speaker != 'agent'].index)
  #preprocessing & classification
    df = cleanText(df)
    df["cat"]= df['transcripts_cleaning'].apply(lambda text:getcat(text)["cat"])
    df["subcat"]= df['transcripts_cleaning'].apply(lambda text:getcat(text)["subcat"])
    df = ending(df)
    df = listening(df)
    df.to_excel(f'/content/drive/MyDrive/iiv/menu2/{file_name[i]}.xlsx', index = False, header=True)
  except:
    print('error',file_name[i])
    error.append(file_name[i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


error 108.082-2185393
error 27.098-2514616


# เตรียมข้อมูลเพื่อเทรนโมเดล

## import data (มี label แล้ว)

นำdataframeแต่ละอันมาconcatกัน

In [ ]:
folder_path1 = '/content/drive/MyDrive/iiv/trainmodel'
file_path1 = walk_csv(folder_path1)

In [ ]:
file_path1

['/content/drive/MyDrive/iiv/trainmodel/1.csv',
 '/content/drive/MyDrive/iiv/trainmodel/10.csv',
 '/content/drive/MyDrive/iiv/trainmodel/11.csv',
 '/content/drive/MyDrive/iiv/trainmodel/12.csv',
 '/content/drive/MyDrive/iiv/trainmodel/13.csv',
 '/content/drive/MyDrive/iiv/trainmodel/14.csv',
 '/content/drive/MyDrive/iiv/trainmodel/15.csv',
 '/content/drive/MyDrive/iiv/trainmodel/16.csv',
 '/content/drive/MyDrive/iiv/trainmodel/17.csv',
 '/content/drive/MyDrive/iiv/trainmodel/18.csv',
 '/content/drive/MyDrive/iiv/trainmodel/19.csv',
 '/content/drive/MyDrive/iiv/trainmodel/2.csv',
 '/content/drive/MyDrive/iiv/trainmodel/20.csv',
 '/content/drive/MyDrive/iiv/trainmodel/21.csv',
 '/content/drive/MyDrive/iiv/trainmodel/22.csv',
 '/content/drive/MyDrive/iiv/trainmodel/23.csv',
 '/content/drive/MyDrive/iiv/trainmodel/24.csv',
 '/content/drive/MyDrive/iiv/trainmodel/25.csv',
 '/content/drive/MyDrive/iiv/trainmodel/26.csv',
 '/content/drive/MyDrive/iiv/trainmodel/27.csv',
 '/content/drive/MyDri

In [ ]:
list_df_concat = []

for i in file_path1 :
  files_list_text1 = i[len(file_path1)+16:-4]
  print(files_list_text1)
  df_file1 = pd.read_csv(i,index_col=None, header=0)
  list_df_concat.append(df_file1)

In [ ]:
list_df_concat[0]

,start,end,speaker,gender,transcripts,transcripts_cleaning,cat,subcat
0,1.568493,5.125556,agent,female,สวัสดีค่ะวรัสรินยินดีให้บริการค่ะ,สวัสดีค่ะวรัสรินยินดีให้บริการค่ะ,Script,Greeting
1,15.898399,17.334969,agent,female,รถทะเบียนอะไรคะ,รถทะเบียนอะไรคะ,Service Dialogue,Agent Inquiry
2,27.652613,29.398318,agent,female,ของคุณน้ำฝนนะคะ,ของคุณน้ำฝนนะคะ,Service Dialogue,Agent Inquiry
3,32.481589,34.072727,agent,female,เป็นมิทซุนะคะอันนี้,เป็นมิทซุนะคะอันนี้,Service Dialogue,Agent Inquiry
4,36.462969,39.190633,agent,female,ค่ะก็คือเหยียบโดนอะไรนะครับโทษค่ะ,ค่ะก็คือเหยียบโดนอะไรนะครับโทษค่ะ,Service Dialogue,Quality
5,58.944982,64.127544,agent,female,แม็กแตกยางแตกก็คือไม่แน่ใจว่าโดนอะไรด้วยใช่ไหมคะ,แม็กแตกยางแตกก็คือไม่แน่ใจว่าโดนอะไรด้วยใช่ไหมคะ,Service Dialogue,Quality
6,66.232898,67.269411,agent,female,เป็นเป็นฝั่งไหนอะคะ,เป็นเป็นฝั่งไหนอะคะ,Service Dialogue,Quality
7,75.009917,80.556168,agent,female,แม็กหน้าขวาแตกแล้วก็ยางหน้าขวาแตกเกิดเหตุอยู่ต...,แม็กหน้าขวาแตกแล้วก็ยางหน้าขวาแตกเกิดเหตุอยู่ต...,Service Dialogue,Quality
8,91.083944,92.275024,agent,female,เกิดเหตุอยู่ที่ไหนคะ,เกิดเหตุอยู่ที่ไหนคะ,Service Dialogue,Quality
9,105.047561,107.402444,agent,female,ขาเข้ากอมอเก้าผู้ขับขี่,ขาเข้ากอมอเก้าผู้ขับขี่,Service Dialogue,Repeat Information


In [ ]:
df_for_model = pd.concat(list_df_concat)

In [ ]:
df_for_model['cat_subcat'] = df_for_model.cat.str.cat(df_for_model.subcat,sep=",")

In [ ]:
df_for_model

,start,end,speaker,gender,transcripts,transcripts_cleaning,cat,subcat,Column1,cat_subcat
0,1.568493,5.125556,agent,female,สวัสดีค่ะวรัสรินยินดีให้บริการค่ะ,สวัสดีค่ะวรัสรินยินดีให้บริการค่ะ,Script,Greeting,NaN,"Script,Greeting"
1,15.898399,17.334969,agent,female,รถทะเบียนอะไรคะ,รถทะเบียนอะไรคะ,Service Dialogue,Agent Inquiry,NaN,"Service Dialogue,Agent Inquiry"
2,27.652613,29.398318,agent,female,ของคุณน้ำฝนนะคะ,ของคุณน้ำฝนนะคะ,Service Dialogue,Agent Inquiry,NaN,"Service Dialogue,Agent Inquiry"
3,32.481589,34.072727,agent,female,เป็นมิทซุนะคะอันนี้,เป็นมิทซุนะคะอันนี้,Service Dialogue,Agent Inquiry,NaN,"Service Dialogue,Agent Inquiry"
4,36.462969,39.190633,agent,female,ค่ะก็คือเหยียบโดนอะไรนะครับโทษค่ะ,ค่ะก็คือเหยียบโดนอะไรนะครับโทษค่ะ,Service Dialogue,Quality,NaN,"Service Dialogue,Quality"
...,...,...,...,...,...,...,...,...,...,...
30,199.948128,206.703850,agent,female,ตราดฝั่งขาออกค่ะก็คือเข้าซอยไปแล้วจะรอเข้าซอยว...,ตราดฝั่งขาออกค่ะก็คือเข้าซอยไปแล้วจะรอเข้าซอยว...,Service Dialogue,Agent Inquiry,NaN,"Service Dialogue,Agent Inquiry"
31,208.329024,214.734708,agent,female,ค่ะเป็นพรุ่งนี้เก้าโมงเช้าเดี๋ยวสุมิตาดูตารางพ...,ค่ะเป็นพรุ่งนี้เก้าโมงเช้าเดี๋ยวสุมิตาดูตารางพ...,Service Dialogue,Agent Response,NaN,"Service Dialogue,Agent Response"
32,218.149507,222.609983,agent,female,ค่ะได้ค่ะเดี๋ยวประสานงานทางเจ้าหน้าที่ไปพบพรุ่...,ค่ะได้ค่ะเดี๋ยวประสานงานทางเจ้าหน้าที่ไปพบพรุ่...,Service Dialogue,Agent Response,NaN,"Service Dialogue,Agent Response"
33,224.100709,231.390866,agent,female,ที่วัดศรีวารีน้อยค่ะสุมิตารับเรื่องไว้ค่ะพรุ่ง...,ที่วัดศรีวารีน้อยค่ะสุมิตารับเรื่องไว้ค่ะพรุ่ง...,Script,Before Ending,NaN,"Script,Before Ending"


In [ ]:
df_for_model.columns

Index(['start', 'end', 'speaker', 'gender', 'transcripts',
       'transcripts_cleaning', 'cat', 'subcat', 'Column1', 'cat_subcat'],
      dtype='object')

column: translate_cleaning มีไว้เพื่อสะดวกต่อการทำ label ในขั้นตอนก่อนหน้านี้เท่านั้น ตอนนี้ไม่จำเป็นจึงตัดออก

In [ ]:
df_for_model.drop(['start','end','speaker','transcripts_cleaning','cat','subcat','Column1'],axis=1,inplace=True)
df_for_model

,gender,transcripts,cat_subcat
0,female,สวัสดีค่ะวรัสรินยินดีให้บริการค่ะ,"Script,Greeting"
1,female,รถทะเบียนอะไรคะ,"Service Dialogue,Agent Inquiry"
2,female,ของคุณน้ำฝนนะคะ,"Service Dialogue,Agent Inquiry"
3,female,เป็นมิทซุนะคะอันนี้,"Service Dialogue,Agent Inquiry"
4,female,ค่ะก็คือเหยียบโดนอะไรนะครับโทษค่ะ,"Service Dialogue,Quality"
...,...,...,...
30,female,ตราดฝั่งขาออกค่ะก็คือเข้าซอยไปแล้วจะรอเข้าซอยว...,"Service Dialogue,Agent Inquiry"
31,female,ค่ะเป็นพรุ่งนี้เก้าโมงเช้าเดี๋ยวสุมิตาดูตารางพ...,"Service Dialogue,Agent Response"
32,female,ค่ะได้ค่ะเดี๋ยวประสานงานทางเจ้าหน้าที่ไปพบพรุ่...,"Service Dialogue,Agent Response"
33,female,ที่วัดศรีวารีน้อยค่ะสุมิตารับเรื่องไว้ค่ะพรุ่ง...,"Script,Before Ending"


## word_tokenize

In [ ]:
from pythainlp.corpus.common import thai_stopwords
from pythainlp import word_tokenize
thai_stopwords = list(thai_stopwords())
thai_stopwords

def text_process(text):
    final = "".join(u for u in text if u not in ("?", ".", ";", ":", "!", '"', "ๆ", "ฯ"))
    final = word_tokenize(final)
    final = " ".join(word for word in final)
    final = " ".join(word for word in final.split() 
              if word.lower not in thai_stopwords)
    return final
df_for_model['text_tokens'] = df_for_model['transcripts'].apply(text_process)
df_for_model

,gender,transcripts,cat_subcat,text_tokens
0,female,สวัสดีค่ะวรัสรินยินดีให้บริการค่ะ,"Script,Greeting",สวัสดี ค่ะ ว รัส ริน ยินดี ให้บริการ ค่ะ
1,female,รถทะเบียนอะไรคะ,"Service Dialogue,Agent Inquiry",รถ ทะเบียน อะไร คะ
2,female,ของคุณน้ำฝนนะคะ,"Service Dialogue,Agent Inquiry",ของ คุณ น้ำฝน นะคะ
3,female,เป็นมิทซุนะคะอันนี้,"Service Dialogue,Agent Inquiry",เป็น มิ ทซุ นะคะ อันนี้
4,female,ค่ะก็คือเหยียบโดนอะไรนะครับโทษค่ะ,"Service Dialogue,Quality",ค่ะ ก็ คือ เหยียบ โดน อะไร นะ ครับ โทษ ค่ะ
...,...,...,...,...
30,female,ตราดฝั่งขาออกค่ะก็คือเข้าซอยไปแล้วจะรอเข้าซอยว...,"Service Dialogue,Agent Inquiry",ตราด ฝั่ง ขาออก ค่ะ ก็ คือ เข้า ซอย ไป แล้ วจะ...
31,female,ค่ะเป็นพรุ่งนี้เก้าโมงเช้าเดี๋ยวสุมิตาดูตารางพ...,"Service Dialogue,Agent Response",ค่ะ เป็น พรุ่งนี้ เก้า โมง เช้า เดี๋ยว สุ มิ ต...
32,female,ค่ะได้ค่ะเดี๋ยวประสานงานทางเจ้าหน้าที่ไปพบพรุ่...,"Service Dialogue,Agent Response",ค่ะ ได้ ค่ะ เดี๋ยว ประสานงาน ทาง เจ้าหน้าที่ ไ...
33,female,ที่วัดศรีวารีน้อยค่ะสุมิตารับเรื่องไว้ค่ะพรุ่ง...,"Script,Before Ending",ที่วัด ศรี วารี น้อย ค่ะ สุ มิ ตา รับ เรื่อง ไ...


In [ ]:
df_for_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2322 entries, 0 to 34
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   gender       2322 non-null   object
 1   transcripts  2322 non-null   object
 2   cat_subcat   2322 non-null   object
 3   text_tokens  2322 non-null   object
dtypes: object(4)
memory usage: 90.7+ KB


## Encoding


In [ ]:
from sklearn.model_selection import train_test_split
X = df_for_model[['text_tokens']]
y = df_for_model['cat_subcat']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
#ความถี่ของคำ
from sklearn.feature_extraction.text import CountVectorizer
cvec = CountVectorizer(analyzer=lambda x:x.split(' '))
cvec.fit_transform(X_train['text_tokens'])
cvec.vocabulary_

{'ได้': 1546,
 'ค่ะ': 154,
 'พันธ์': 656,
 'นี้': 498,
 'พอ': 646,
 'พัน': 655,
 'ไม้': 1565,
 'หัน': 1093,
 'นอ': 464,
 'หนู': 1052,
 'การันต์': 45,
 'ใช่ไหม': 1523,
 'คะ': 130,
 'เป็น': 1293,
 'รถ': 763,
 'ยี่ห้อ': 748,
 'โตโยต้า': 1487,
 'ประมาณ': 568,
 'บ่ายโมง': 544,
 'ไหม': 1573,
 'คุณ': 143,
 'ชาญ': 221,
 'ชัย': 218,
 'ยาง': 739,
 'ล้อ': 863,
 'ไหน': 1572,
 'ฉีก': 211,
 'เรียบร้อย': 1340,
 'ปา': 585,
 'รี': 808,
 'ณา': 266,
 'รับ': 792,
 'เรื่อง': 1342,
 'ไว้': 1570,
 'ขอ': 66,
 'อนุ': 1110,
 'ญาติ': 255,
 'ให้': 1538,
 'เจ้าหน้าที่': 1229,
 'ติดต่อ': 333,
 'กลับ': 20,
 'ด้วย': 286,
 'นะคะ': 469,
 'เพื่อ': 1315,
 'ติด': 331,
 'ตามเรื่อง': 326,
 'ว่า': 914,
 'มี': 711,
 'พนักงาน': 625,
 'ไป': 1553,
 'ทำ': 401,
 'เคลม': 1211,
 'หรือยัง': 1074,
 'ร้าน': 824,
 'เอ๋': 1405,
 'สโตร์': 1029,
 'ตอน': 309,
 'เกิดเหตุ': 1194,
 'คน': 99,
 'ขับขี่': 81,
 'ชื่อ': 224,
 'อะไร': 1134,
 'เรา': 1336,
 'ปริ': 577,
 'ญญ์': 252,
 'เก้า': 1198,
 'ศูนย์': 930,
 'ห้า': 1101,
 'ฝาครอบ': 618,
 'ล้อหน้า'

In [ ]:
train_bow = cvec.transform(X_train['text_tokens'])
pd.DataFrame(train_bow.toarray(), columns=cvec.get_feature_names(), index=X_train['text_tokens'])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,-,11,ก,กด,กตศ,กน,กนก,กบ,กร,กรก,...,ไว้,ไห,ไหน,ไหม,ไอ,ไอยรา,ไฮ,ไฮลักซ์,้,์
text_tokens,,,,,,,,,,,,,,,,,,,,,
ได้ ค่ะ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
พันธ์ นี้ พอ พัน ไม้ หัน นอ หนู การันต์ ใช่ไหม คะ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ค่ะ เป็น รถ ยี่ห้อ โตโยต้า,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ได้ ได้ ประมาณ บ่ายโมง ได้ ไหม คะ คุณ ชาญ ชัย,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
ยาง ล้อ ไหน ฉีก คะ,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
คุณ นัน หญิง ตอนนี้ เรียน สาย อยู่ กับ คุณ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
วันนี้ บ่ายโมง ครึ่ง วัน ที่หนึ่ง ค่ะ ช่วง บ่ายโมง ครึ่ง คุณ ศ รา วุฒิ ยังอยู่ ที่ บริษัท ไหม คะ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
ค่ะ แล้ว ตอน เกิดเหตุ คุณ อะไร เป็น ผู้ขับขี่ รถ เรา คะ ตอน เกิดเหตุ อะ ค่ะ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Model : LogisticRegression

In [ ]:
print(X_train.shape)
print(X_test.shape)

(1625, 1)
(697, 1)


In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
my_predictions=lr.fit(train_bow, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
X_test

,text_tokens
19,ที่ เสีย ตัง ลูกค้า ต้อง เสีย ตัง ไป ก่อน แล้ว...
22,ไม่ มี ผู้บาดเจ็บ ใช่ไหม คะ คุณ สุทธิ ชัย
6,ค่ะ เดี๋ยว รบกวน ถือ สาย รอ สัก ครุ่ นะคะ
52,ใช่
13,หอพัก อยู่ แถว ถนน อะไร คะ
...,...
36,ขอบคุณ ค่ะ
3,เป็น รถ ยี่ห้อ อีซูซุ ดีแมกซ์ นะคะ
50,ศูนย์ แปด เก้า ค่ะ
14,ครับ


In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
test_bow = cvec.transform(X_test['text_tokens'])
test_predictions = lr.predict(test_bow)
print(classification_report(test_predictions, y_test))

                                     precision    recall  f1-score   support

               Script,Before Ending       0.54      0.88      0.67        17
                      Script,Ending       0.81      0.85      0.83        26
                    Script,Greeting       0.96      0.96      0.96        26
            Script,Transfer service       0.00      0.00      0.00         1
           Script,offer of benefits       0.00      0.00      0.00         0
     Service Dialogue,Agent Inquiry       0.80      0.73      0.76       223
    Service Dialogue,Agent Response       0.69      0.81      0.75        70
         Service Dialogue,Listening       0.88      0.70      0.78        87
           Service Dialogue,Meeting       0.44      0.73      0.55        15
           Service Dialogue,Quality       0.73      0.80      0.76       129
Service Dialogue,Repeat Information       0.76      0.66      0.70       103
          Service Dialogue,Transfer       0.00      0.00      0.00         

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

In [ ]:
X_test['cat-subcat'] = test_predictions

#แบ่งคอลัมน์
X_test[['Cat','Subcat']] = X_test['cat-subcat'].str.split(',', expand=True)
#ลบคอลัมน์ cat-subcat
X_test.drop(['cat-subcat'],axis=1,inplace=True)

In [ ]:
X_test

,text_tokens,Cat,Subcat
19,ที่ เสีย ตัง ลูกค้า ต้อง เสีย ตัง ไป ก่อน แล้ว...,Service Dialogue,Agent Response
22,ไม่ มี ผู้บาดเจ็บ ใช่ไหม คะ คุณ สุทธิ ชัย,Service Dialogue,Agent Inquiry
6,ค่ะ เดี๋ยว รบกวน ถือ สาย รอ สัก ครุ่ นะคะ,Script,Transfer service
52,ใช่,Service Dialogue,Listening
13,หอพัก อยู่ แถว ถนน อะไร คะ,Service Dialogue,Agent Inquiry
...,...,...,...
36,ขอบคุณ ค่ะ,Service Dialogue,Listening
3,เป็น รถ ยี่ห้อ อีซูซุ ดีแมกซ์ นะคะ,Service Dialogue,Agent Inquiry
50,ศูนย์ แปด เก้า ค่ะ,Service Dialogue,Repeat Information
14,ครับ,Service Dialogue,Listening


In [ ]:
X_test.to_csv("cat_subcat.csv")

# Test

In [ ]:
my_text = 'สวัสดีค่ะ วิริยะประกันภัยยินดีให้บริการค่ะ'
my_tokens = text_process(my_text)
my_bow = cvec.transform(pd.Series([my_tokens]))
my_predictions = lr.predict(my_bow)
my_predictions

array(['Script,Greeting'], dtype=object)

# Convert to onnx

In [ ]:
import numpy
pred = lr.predict(test_bow)
print(confusion_matrix(y_test, pred))

[[ 15   2   0   0   0   2   5   1   1   0   2   0]
 [  1  22   1   0   0   0   0   2   0   1   0   0]
 [  0   1  25   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   1   0   0   0   2   0]
 [  0   0   0   0   0 162   3   7   1  19  10   0]
 [  1   0   0   1   0  10  57   4   2   4   4   0]
 [  0   0   0   0   0   4   0  61   0   0   4   0]
 [  0   1   0   0   0   6   1   1  11   1   4   0]
 [  0   0   0   0   0  31   0   0   0 103   8   0]
 [  0   0   0   0   0   8   3  10   0   1  68   0]
 [  0   0   0   0   0   0   0   1   0   0   1   0]]


In [ ]:
!pip install skl2onnx
!pip install onnxruntime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 278 kB 4.4 MB/s 
     |████████████████████████████████| 78 kB 7.5 MB/s 
     |████████████████████████████████| 12.8 MB 53.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.2 MB 5.0 MB/s 


In [ ]:
# We use module sklearn-onnx to convert the model into ONNX format.

from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(lr, initial_types=initial_type)
with open("logreg_cat_subcat.onnx", "wb") as f:
    f.write(onx.SerializeToString())

In [ ]:
# We load the model with ONNX Runtime and look at its input and output.

import onnxruntime as rt
sess = rt.InferenceSession("logreg_cat_subcat.onnx", providers=rt.get_available_providers())

print("input name='{}' and shape={}".format(
    sess.get_inputs()[0].name, sess.get_inputs()[0].shape))
print("output name='{}' and shape={}".format(
    sess.get_outputs()[0].name, sess.get_outputs()[0].shape))

input name='float_input' and shape=[None, 4]
output name='output_label' and shape=[None]


In [ ]:
# We compute the predictions.
import numpy
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

pred_onx = sess.run([label_name], {input_name: float(test_bow).astype(numpy.float32)})[0]
print(confusion_matrix(pred, pred_onx))

TypeError: ignored

In [ ]:
test_bow